# Data Cleaning and Set up

### Importing Libraries

In [15]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import random

from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

# Disable warnings
import warnings
warnings.filterwarnings("ignore")

### Data Inspection

In [2]:
base_df = pd.read_csv('historical_data.csv')
base_df['created_at'] = pd.to_datetime(base_df['created_at'])
base_df['actual_delivery_time'] = pd.to_datetime(base_df['actual_delivery_time'])

base_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197428 entries, 0 to 197427
Data columns (total 16 columns):
 #   Column                                        Non-Null Count   Dtype         
---  ------                                        --------------   -----         
 0   market_id                                     196441 non-null  float64       
 1   created_at                                    197428 non-null  datetime64[ns]
 2   actual_delivery_time                          197421 non-null  datetime64[ns]
 3   store_id                                      197428 non-null  int64         
 4   store_primary_category                        192668 non-null  object        
 5   order_protocol                                196433 non-null  float64       
 6   total_items                                   197428 non-null  int64         
 7   subtotal                                      197428 non-null  int64         
 8   num_distinct_items                            197428 n

### Dealing with Null Values

In [3]:
# Null count for each column
print(base_df.isnull().sum())

market_id                                         987
created_at                                          0
actual_delivery_time                                7
store_id                                            0
store_primary_category                           4760
order_protocol                                    995
total_items                                         0
subtotal                                            0
num_distinct_items                                  0
min_item_price                                      0
max_item_price                                      0
total_onshift_dashers                           16262
total_busy_dashers                              16262
total_outstanding_orders                        16262
estimated_order_place_duration                      0
estimated_store_to_consumer_driving_duration      526
dtype: int64


In [6]:
# Dropping rows with missing information on target variable
df = base_df.dropna(subset=['actual_delivery_time'])

# Replace nan with 'other' on store_primary_category
df['store_primary_category'] = df['store_primary_category'].fillna('other')

# Dropping nan values on order_protocol and market_id
df = df.dropna(subset=['order_protocol','market_id'])

# Target variable
df['target_variable'] = (df['actual_delivery_time'] - df['created_at']).dt.total_seconds()

# Day of the week, hour of the day and month of the year from created_at
df['day'] = df.created_at.dt.day_of_week
df['hour'] = df.created_at.dt.hour

In [8]:
# Drop outliers based on 0.25 and 0.75 quantiles on sub_total, target_variable and max_item_price
print('Length before removing outliers: ', len(df))

def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    return df[(df[column] >= Q1 - 1.5 * IQR) & (df[column] <= Q3 + 1.5 * IQR)]

df = remove_outliers(df, 'subtotal')
df = remove_outliers(df, 'target_variable')
df = remove_outliers(df, 'max_item_price')

print('Length after removing outliers: ', len(df))

Length before removing outliers:  195926
Length after removing outliers:  174251


In [9]:
# KNN imputation for missing values on total_onshift_dashers, total_busy_dashers, total_outstanding_orders

imputer = KNNImputer(n_neighbors=50)
imputation_columns = ['total_onshift_dashers','total_busy_dashers','total_outstanding_orders']
reference_columns = ['day','hour','market_id']
impute_df = df[imputation_columns+reference_columns]
imputed_values = imputer.fit_transform(impute_df)

df['total_onshift_dashers'] = imputed_values[:,0]
df['total_busy_dashers'] = imputed_values[:,1]
df['total_outstanding_orders'] = imputed_values[:,2]

In [10]:
# Final touch up on missing values

# Dropping rows with missing values on estimated_store_to_consumer_driving_duration
df = df.dropna(subset=['estimated_store_to_consumer_driving_duration'])

fill_df = df.copy()

In [11]:
# Storing fill_df to a csv file
fill_df.to_csv('filled_data.csv', index=False)

In [12]:
# Restore fill_df from csv file if necessary
# fill_df = pd.read_csv('filled_data.csv')

### Encoding Categorical Variables

In [13]:
fill_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 173774 entries, 0 to 197427
Data columns (total 19 columns):
 #   Column                                        Non-Null Count   Dtype         
---  ------                                        --------------   -----         
 0   market_id                                     173774 non-null  float64       
 1   created_at                                    173774 non-null  datetime64[ns]
 2   actual_delivery_time                          173774 non-null  datetime64[ns]
 3   store_id                                      173774 non-null  int64         
 4   store_primary_category                        173774 non-null  object        
 5   order_protocol                                173774 non-null  float64       
 6   total_items                                   173774 non-null  int64         
 7   subtotal                                      173774 non-null  int64         
 8   num_distinct_items                            173774 non-nu

In [17]:
cat_df = fill_df.copy()

# Dropping columns that are not useful for the model
# cat_df.drop(['store_id'], axis=1, inplace=True)

# One-hot encoding on market_id and order_protocol
cat_df = pd.get_dummies(cat_df, columns=['market_id','order_protocol', 'day'])

# Label encoding on store_primary_category
cat_df_label = cat_df.copy()

label_encoder = LabelEncoder()
cat_df_label['store_primary_category'] = label_encoder.fit_transform(cat_df['store_primary_category'])

# One hot encoding of store_primary_category
cat_df_one = pd.get_dummies(cat_df, columns=['store_primary_category'])


In [18]:
# Storing cat_df_one to a csv file
cat_df_one.to_csv('df_clean_one.csv', index=False)

# Storing cat_df_label to a csv file
cat_df_label.to_csv('df_clean_label.csv', index=False)